In [1]:
"a"

'a'

In [4]:
from sqlalchemy.ext.asyncio import async_sessionmaker, create_async_engine, AsyncSession
from sqlalchemy import create_engine
from sqlalchemy.sql import text

DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "itam_messenger"
DB_USER ="postgres"
DB_PASSWORD = "postgres"


DATABASE_URL=f"postgresql+asyncpg://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}" 
engine=None

engine = create_async_engine(url=DATABASE_URL)
sessionmaker = async_sessionmaker(bind=engine, expire_on_commit=False)
async with sessionmaker() as session:
    a = await session.execute(text('SELECT * FROM meow'))
    print(a)
 

ProgrammingError: (sqlalchemy.dialects.postgresql.asyncpg.ProgrammingError) <class 'asyncpg.exceptions.UndefinedTableError'>: relation "meow" does not exist
[SQL: SELECT * FROM meow]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [55]:
async with engine.connect() as conection:
    print(conection)

In [57]:
from sqlalchemy import Text, ForeignKey, DateTime, BigInteger, Column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import declarative_base

DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "postgres"
DB_USER ="postgres"
DB_PASSWORD = "postgres"

DATABASE_URL=f"postgresql+asyncpg://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

Base = declarative_base()
engine = create_engine(url=DATABASE_URL)

async with engine.connect() as conection:
    print(conection)

class User(Base):
    __tablename__ = "user"
    id = Column(BigInteger, primary_key=True, nullable=False, autoincrement=True)
    username = Column(Text, nullable=False)
    short_username = Column(Text)
    status = Column(Text)
    photo = Column(Text) #text_link
    password = Column(Text, nullable=False) #text_hashasync with engine.connect() as conection:
    print(conection)
    created_at = Column(DateTime, nullable=False)

class Message(Base):
    __tablename__ = "message"
    id = Column(BigInteger, primary_key=True, nullable=False, autoincrement=True)
    chat_id = Column(BigInteger, ForeignKey("chat.id"), nullable=False)
    user_id = Column(BigInteger, ForeignKey("user.id"), nullable=False)
    created_at = Column(DateTime, nullable=False)

class Chat(Base):
    __tablename__ = "chat"
    id = Column(BigInteger, primary_key=True, nullable=False, autoincrement=True)
    chat_name = Column(Text)
    photo = Column(Text) # photo link
    created_at = Column(DateTime, nullable=False)


class Member(Base):
    __tablename__ = "member"
    id = Column(BigInteger, autoincrement=True, primary_key=True, nullable=False)
    chat_id = Column(BigInteger, ForeignKey("chat.id"), nullable=False)
    user_id = Column(BigInteger, ForeignKey("user.id"), nullable=False)
    added_at = Column(DateTime, nullable=False)
    role = Column(Text) # owner, admin, plain

Base.metadata.create_all(engine)




MissingGreenlet: greenlet_spawn has not been called; can't call await_only() here. Was IO attempted in an unexpected place? (Background on this error at: https://sqlalche.me/e/20/xd2s)

In [2]:
from sqlalchemy import Text, ForeignKey, DateTime, BigInteger, Column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import declarative_base

In [8]:
DB_HOST = "127.0.0.1"
DB_PORT = "5432"
DB_NAME = "itam_messenger"
DB_USER ="postgres"
DB_PASSWORD = "postgres"
DATABASE_URL_ASYNC=f"postgresql+asyncpg://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
DATABASE_URL_SYNC=f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

Base = declarative_base()

engine = create_async_engine(url=DATABASE_URL_ASYNC)
try:
    async with engine.connect() as connection:
        #result = await connection.execute(text("SELECT * FROM meow"))
        print("Подключение к базе данных успешно!")
        #print(result.fetchone())
except Exception as e:
    print("Ошибка подключения к базе данных:")
    print(e)

engine = create_engine(url=DATABASE_URL_SYNC)
try:
    with engine.connect() as connection:
        #result = connection.execute(text("SELECT * FROM meow"))
        print("Подключение к базе данных успешно!")
        #print(result.fetchone())
except Exception as e:
    print("Ошибка подключения к базе данных:")
    print(e)

Подключение к базе данных успешно!
Подключение к базе данных успешно!


In [4]:
### s3 selectel api
import asyncio

In [23]:
from aiobotocore.session import get_session
from contextlib import asynccontextmanager
import uuid

class S3Client:
    def __init__(
        self,
        access_key: str,
        secret_key: str,
        endpoint_url: str,
        bucket_name:str 
    ):
        self.config = {
            "aws_access_key_id": access_key,
            "aws_secret_access_key": secret_key,
            "endpoint_url": endpoint_url
        }
        self.bucket_name = bucket_name
        self.session = get_session()

    @asynccontextmanager
    async def get_client(self):
        async with self.session.create_client("s3", **self.config) as client:
            yield client

    async def upload_file(
        self, 
        file_path: str,
        object_name: str="a",
        user_id: str=1

    ):
        object_name = file_path.split("/")[-1] #+ str(uuid.uuid4()) + str(user_id)
        async with self.get_client() as client:
            with open(file_path, "rb") as file:
                await client.put_object(
                    Bucket=self.bucket_name,
                    Key=object_name,
                    Body=file
                )

    async def get_file(self, object_name: str, destination_path: str):
        """Позволяет получить файл."""
        try:
            async with self.get_client() as client:
                response = await client.get_object(
                    Bucket=self.bucket_name, Key=object_name
                )
                data = await response['Body'].read()
                with open(destination_path, 'wb') as file:
                    file.write(data)
                print(f'File {object_name} has been downloaded'
                      f'to {destination_path}')
        except ClientError as error:
            print(f'Error while downloading file: {error}')




async def main():
    s3_client = S3Client(
        access_key="55e1ffce5ecf4210b6717ceb9cabbf87",
        secret_key="bb08bfb363bb41af9463aed33e2693a6",
        endpoint_url="https://s3.ru-1.storage.selcloud.ru",
        bucket_name="afina-storage"
    )
    
    await s3_client.upload_file("debug.ipynb")
    await s3_client.get_file("debug.ipynb", "text_local_file.txt")
    

await main()

File debug.ipynb has been downloadedto text_local_file.txt


In [10]:
from aiobotocore.session import get_session
from contextlib import asynccontextmanager
import uuid

class S3Client:
    def __init__(
        self,
        access_key: str,
        secret_key: str,
        endpoint_url: str,
        bucket_name:str 
    ):
        self.config = {
            "aws_access_key_id": access_key,
            "aws_secret_access_key": secret_key,
            "endpoint_url": endpoint_url
        }
        self.bucket_name = bucket_name
        self.session = get_session()

    @asynccontextmanager
    async def get_client(self):
        """Создает подключение к S3 хранилищу."""
        async with self.session.create_client('s3', **self.config) as client:
            yield client

    async def upload_file(
        self, 
        file_path: str,
        object_name: str
    ):
        try:
            async with self.get_client() as client:
                with open(file_path, "rb") as file:
                    await client.put_object(
                        Bucket=self.bucket_name,
                        Key=object_name,
                        Body=file
                    )
        except ClientError as error:
            print(f'Error while uploading a file: {error}') 

    async def get_file(self, object_name: str, destination_path: str):
        async with self.get_client() as client:
            response = await client.get_object(
                Bucket=self.bucket_name,
                Key=object_name
            )
            data = await response["Body"].read()
            with open(destination_path, 'wb') as file:
                file.write(data)
            print(f'File {object_name} has been downloaded'
                    f'to {destination_path}')


    async def delete_file(self, object_name: str):
        """Позволяет удалить файл."""
        try:
            async with self.get_client() as client:
                await client.delete_object(
                    Bucket=self.bucket_name,
                    Key=object_name
                )
                print(f'File {object_name} has been deleted'
                      f'from {self.bucket_name}')
        except ClientError as error:
            print(f'Error while deleting file: {error}')


async def main():
    s3_client = S3Client(
        access_key="55e1ffce5ecf4210b6717ceb9cabbf87",
        secret_key="bb08bfb363bb41af9463aed33e2693a6",
        endpoint_url="https://s3.ru-1.storage.selcloud.ru",
        bucket_name="afina-storage"
    )
    
    file_path = "/home/mirkuriit/misis/itam/messenger_proj_beckend/settings/settings.py"
    file_uuid4 = uuid.uuid4()
    object_name = ">".join([str(file_uuid4), file_path.split("/")[-1]])
    #print(object_name)
    #await s3_client.upload_file(file_path=file_path, object_name=object_name)
    await s3_client.get_file(object_name="0c66d19b-67ac-4dde-bf8d-0169b85444c4>settings.py", destination_path="meow")
    

await main()

File 0c66d19b-67ac-4dde-bf8d-0169b85444c4>settings.py has been downloadedto meow


In [18]:
async def main():
    s3_client = S3Client(
        access_key="55e1ffce5ecf4210b6717ceb9cabbf87",
        secret_key="bb08bfb363bb41af9463aed33e2693a6",
        endpoint_url="https://s3.ru-1.storage.selcloud.ru",
        bucket_name="afina-storage"
    )
    
    await s3_client.upload_file("debug.ipynb")
    await s3_client.get_file("debug.ipynb", "text_local_file.txt")
    

await main()

In [7]:
! ls 

debug.ipynb	    main.py	presentation	  services
infrastructure	    persistent	repository	  settings
itam_messenger_env  pg_db	requirements.txt  text_local_file.txt


In [12]:
! ls

debug.ipynb	    main.py	presentation	  services
infrastructure	    persistent	repository	  settings
itam_messenger_env  pg_db	requirements.txt
